In [204]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime
import calendar

## 1.1 EU Data

In [323]:
df_eu = pd.read_csv('Data/EU_MILK_trade_data_en.csv')
df_eu.head()

,Flow,Marketing Year,Month,Month Order in MY,Month Date,Member State,Partner,Product Group,Product Code (CN),Quantity in tonnes,Value in thousand euro
0,EXPORT,2022,Oct,10,01/10/2022,Austria,Albania,Cheese,4063039,17.677,86.192
1,EXPORT,2022,Oct,10,01/10/2022,Austria,Angola,Cheese,4061050,0.472,2.189
2,EXPORT,2022,Oct,10,01/10/2022,Austria,Angola,Cheese,4063031,9.862,41.178
3,EXPORT,2022,Oct,10,01/10/2022,Austria,Angola,Cheese,4063039,3.177,14.467
4,EXPORT,2022,Oct,10,01/10/2022,Austria,Antigua and Barbuda,Cheese,4063031,1.800,9.212


In [324]:
df_eu = df_eu[(df_eu['Product Code (CN)'].astype('string').str[:5]=='40510') & (df_eu['Flow']=='EXPORT')]

In [325]:
df_eu.isna().sum()

Flow                      0
Marketing Year            0
Month                     0
Month Order in MY         0
Month Date                0
Member State              0
Partner                   0
Product Group             0
Product Code (CN)         0
Quantity in tonnes        0
Value in thousand euro    0
dtype: int64

In [326]:
df_eu['Product Group'].unique()

array(['Butter', 'Butteroil'], dtype=object)

In [327]:
# Use lambda function to convert the Month Date values to datetime
df_eu['Month Date'] = df_eu['Month Date'].map(lambda x: datetime.strptime(x, '%d/%m/%Y'))


In [328]:
df_eu[(df_eu['Member State']=='Austria')]

,Flow,Marketing Year,Month,Month Order in MY,Month Date,Member State,Partner,Product Group,Product Code (CN),Quantity in tonnes,Value in thousand euro
35,EXPORT,2022,Oct,10,2022-10-01,Austria,Holy See (Vatican City State),Butter,4051011,1.800,14.275
89,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butter,4051011,0.010,0.084
90,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butteroil,4051090,0.000,0.007
130,EXPORT,2022,Oct,10,2022-10-01,Austria,United Arab Emirates,Butter,4051011,0.376,1.564
10165,EXPORT,2022,Sep,9,2022-09-01,Austria,Holy See (Vatican City State),Butter,4051011,1.200,9.535
...,...,...,...,...,...,...,...,...,...,...,...
1459527,EXPORT,2010,Jan,1,2010-01-01,Austria,Russia,Butter,4051011,1.272,6.639
1459553,EXPORT,2010,Jan,1,2010-01-01,Austria,Switzerland,Butter,4051019,0.479,1.242
1459554,EXPORT,2010,Jan,1,2010-01-01,Austria,Switzerland,Butteroil,4051090,0.128,0.465
1459605,EXPORT,2010,Jan,1,2010-01-01,Austria,Turkey,Butteroil,4051090,0.009,0.050


In [329]:
df_eu.rename(columns = {'Marketing Year':'Year', 
                        'Member State':'Country',
                        'Product Code (CN)':'Product',
                        'Quantity in tonnes':'Quantity (tonnes)',
                        'Value in thousand euro':'Value (k Euro)'
                       }, inplace = True)

In [330]:
df_eu.drop(['Month Order in MY','Product Group'], axis=1,inplace=True)

## 1.2 NewZealand Data

In [331]:
nz_files = [
    {
        'filename':'Data/NZ/nz_exports_quantities_01.txt',
        'filenames':['Data/NZ/nz_exports_quantities_01.txt',
                    'Data/NZ/nz_exports_quantities_02.txt',
                    'Data/NZ/nz_exports_quantities_03.txt',
                    'Data/NZ/nz_exports_quantities_04.txt',
                    'Data/NZ/nz_exports_quantities_05.txt',
                    'Data/NZ/nz_exports_quantities_06.txt',
                    'Data/NZ/nz_exports_quantities_07.txt',
                    'Data/NZ/nz_exports_quantities_08.txt'],
        'partner_column':'Importers',
        'column_name':'Quantity in tonnes',
        'flow':'EXPORT',
        'left_date_text':'',
        'right_date_text':'%Y-M%m-Exported quantity, Kilograms',
        'date_format':'%Y-M%m'
    },
    {
        'filename':'Data/NZ/nz_exports_values_01.txt',
        'filenames':['Data/NZ/nz_exports_values_01.txt',
                    'Data/NZ/nz_exports_values_02.txt',
                    'Data/NZ/nz_exports_values_03.txt',
                    'Data/NZ/nz_exports_values_04.txt',
                    'Data/NZ/nz_exports_values_05.txt',
                    'Data/NZ/nz_exports_values_06.txt',
                    'Data/NZ/nz_exports_values_07.txt',
                    'Data/NZ/nz_exports_values_08.txt'],
        'partner_column':'Importers',
        'column_name':'Value in thousand euro',
        'flow':'EXPORT',
        'left_date_text':'Exported value in ',
        'right_date_text':'',
        'date_format':'%Y-M%m'
    },
    {
        'filename':'Data/NZ/nz_imports_quantities_01.txt',
        'filenames':['Data/NZ/nz_imports_quantities_01.txt',
                    'Data/NZ/nz_imports_quantities_02.txt',
                    'Data/NZ/nz_imports_quantities_03.txt',
                    'Data/NZ/nz_imports_quantities_04.txt',
                    'Data/NZ/nz_imports_quantities_05.txt',
                    'Data/NZ/nz_imports_quantities_06.txt',
                    'Data/NZ/nz_imports_quantities_07.txt',
                    'Data/NZ/nz_imports_quantities_08.txt'],
        'partner_column':'Exporters',
        'column_name':'Quantity in tonnes',
        'flow':'IMPORT',
        'left_date_text':'',
        'right_date_text':'-Imported quantity, Kilograms',
        'date_format':'%Y-M%m'
    },
    {
        'filename':'Data/NZ/nz_imports_values_01.txt',
        'filenames':['Data/NZ/nz_imports_values_01.txt',
                    'Data/NZ/nz_imports_values_02.txt',
                    'Data/NZ/nz_imports_values_03.txt',
                    'Data/NZ/nz_imports_values_04.txt',
                    'Data/NZ/nz_imports_values_05.txt',
                    'Data/NZ/nz_imports_values_06.txt',
                    'Data/NZ/nz_imports_values_07.txt',
                    'Data/NZ/nz_imports_values_08.txt'],
        'partner_column':'Exporters',
        'column_name':'Value in thousand euro',
        'flow':'IMPORT',
        'left_date_text':'Imported value in ',
        'right_date_text':'',
        'date_format':'%Y-M%m'
    }
]

In [332]:
first_quantity_data = True
first_value_data = True

partner_column_name = 'Partner'

for index in range(len(nz_files)):
    #df = pd.read_csv(nz_files[index]['filename'], sep='\t', lineterminator='\r')
    first_part = True
    for i in range(len(nz_files[index]['filenames'])):
        df_part = pd.read_csv(nz_files[index]['filenames'][i], sep='\t', lineterminator='\r')
        key = nz_files[index]['partner_column']
        
        # Rename the 'Importers' / 'Exporters' column to 'Partner', in line with the EU data 
        df_part.rename(columns = {key:'Partner'}, inplace = True)
        #df_part.columns.values[key] = 'Partner'
        
        if first_part:
            df_parts = df_part
            first_part = False
        else:
            df_parts = df_parts.merge(df_part, on='Partner',how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
            df_parts.drop(df_parts.filter(regex='^Unnamed').columns,axis=1,inplace=True)
            df_parts = df_parts[(df_parts['Partner'] != '\n')]
                   
    df = df_parts
    
    # Use lambda function to strip unwanted characters from beginning (lstrip) 
    # and end (rstrip) of the Importers values
    df['Partner'] = df['Partner'].map(lambda x: x.lstrip('\n"').rstrip('"'))
    
    # Remove the sum row 'World', the last row (blank) and the the last column (blank)
    df = df[(df['Partner'] != 'World')]
    
    # Reshape the data, creating a row for each monthly observation
    df = df.melt(id_vars=['Partner'], var_name='Month Date', value_name=nz_files[index]['column_name'])
    
    # Add a 'Flow' column to indicate IMPORT or EXPORT
    df['Flow'] = nz_files[index]['flow']
    
    
    # Use lambda function to strip unwanted text and convert Month Date values to datetime
    df['Month Date'] = df['Month Date'].map(lambda x: x.lstrip(nz_files[index]['left_date_text']).rstrip(nz_files[index]['right_date_text']))
    df['Month Date'] = df['Month Date'].map(lambda x: datetime.strptime(x, nz_files[index]['date_format']))
    
    if nz_files[index]['column_name'] == 'Value in thousand euro':
        if first_value_data:
            df_values = df
            first_value_data = False
        else:
            df_values = pd.concat([df_values,df],ignore_index=True)
    else:
        if first_quantity_data:
            df_quantities = df
            first_quantity_data = False
        else:
            df_quantities = pd.concat([df_quantities,df],ignore_index=True)
    
    
df_nz = pd.merge(df_values,
                 df_quantities[['Month Date','Partner','Flow','Quantity in tonnes']],
                 on=['Month Date','Partner','Flow'], 
                 how='left')
df_nz

,Partner,Month Date,Value in thousand euro,Flow,Quantity in tonnes
0,Denmark,2010-01-01,8271.0,EXPORT,4332245.0
1,Russian Federation,2010-01-01,5504.0,EXPORT,2008407.0
2,Morocco,2010-01-01,4296.0,EXPORT,2334257.0
3,China,2010-01-01,3211.0,EXPORT,1578730.0
4,Australia,2010-01-01,4608.0,EXPORT,2219114.0
...,...,...,...,...,...
16978,Fiji,2022-09-01,0.0,IMPORT,0.0
16979,France,2022-09-01,0.0,IMPORT,0.0
16980,New Zealand,2022-09-01,544.0,IMPORT,98542.0
16981,India,2022-09-01,4.0,IMPORT,750.0


In [333]:
df_nz['Year'] = pd.DatetimeIndex(df_nz['Month Date']).year
df_nz['Month'] = pd.DatetimeIndex(df_nz['Month Date']).month
df_nz['Month'] = df_nz['Month'].apply(lambda x: calendar.month_abbr[x])

In [334]:
df_nz.rename(columns = {
                        'Quantity in tonnes':'Quantity (tonnes)',
                        'Value in thousand euro':'Value (k Euro)'
                       }, inplace = True)
df_nz['Quantity (tonnes)'] = df_nz['Quantity (tonnes)'] / 1000

In [335]:
df_nz['Country'] = 'New Zealand'

In [336]:
df_nz

,Partner,Month Date,Value (k Euro),Flow,Quantity (tonnes),Year,Month,Country
0,Denmark,2010-01-01,8271.0,EXPORT,4332.245,2010,Jan,New Zealand
1,Russian Federation,2010-01-01,5504.0,EXPORT,2008.407,2010,Jan,New Zealand
2,Morocco,2010-01-01,4296.0,EXPORT,2334.257,2010,Jan,New Zealand
3,China,2010-01-01,3211.0,EXPORT,1578.730,2010,Jan,New Zealand
4,Australia,2010-01-01,4608.0,EXPORT,2219.114,2010,Jan,New Zealand
...,...,...,...,...,...,...,...,...
16978,Fiji,2022-09-01,0.0,IMPORT,0.000,2022,Sep,New Zealand
16979,France,2022-09-01,0.0,IMPORT,0.000,2022,Sep,New Zealand
16980,New Zealand,2022-09-01,544.0,IMPORT,98.542,2022,Sep,New Zealand
16981,India,2022-09-01,4.0,IMPORT,0.750,2022,Sep,New Zealand


## 1.3 Combine Data


In [337]:
df_eu.drop(columns='Product',axis=1,inplace=True)

df_eu = df_eu.groupby(['Country','Flow','Partner','Month Date','Month','Year']).sum().reset_index()

In [338]:
df_eu.head(3)

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
0,Austria,EXPORT,Azerbaijan,2016-08-01,Aug,2016,7.376,27.301
1,Austria,EXPORT,Bahrain,2011-01-01,Jan,2011,0.043,0.100
2,Austria,EXPORT,Bosnia and Herzegovina,2021-09-01,Sep,2021,0.001,0.003


In [339]:
df_nz = df_nz[['Country','Flow','Partner','Month Date','Month','Year','Quantity (tonnes)','Value (k Euro)']]

In [340]:
df_nz.head(3)

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
0,New Zealand,EXPORT,Denmark,2010-01-01,Jan,2010,4332.245,8271.0
1,New Zealand,EXPORT,Russian Federation,2010-01-01,Jan,2010,2008.407,5504.0
2,New Zealand,EXPORT,Morocco,2010-01-01,Jan,2010,2334.257,4296.0


In [341]:
pd.set_option('display.max_rows', 20)           

In [342]:
df_eu_nz = pd.concat([df_eu,df_nz],ignore_index=True)

In [343]:
df_eu_nz

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
0,Austria,EXPORT,Azerbaijan,2016-08-01,Aug,2016,7.376,27.301
1,Austria,EXPORT,Bahrain,2011-01-01,Jan,2011,0.043,0.100
2,Austria,EXPORT,Bosnia and Herzegovina,2021-09-01,Sep,2021,0.001,0.003
3,Austria,EXPORT,Canada,2020-11-01,Nov,2020,0.000,0.017
4,Austria,EXPORT,China,2013-11-01,Nov,2013,2.000,13.900
...,...,...,...,...,...,...,...,...
69078,New Zealand,IMPORT,Fiji,2022-09-01,Sep,2022,0.000,0.000
69079,New Zealand,IMPORT,France,2022-09-01,Sep,2022,0.000,0.000
69080,New Zealand,IMPORT,New Zealand,2022-09-01,Sep,2022,98.542,544.000
69081,New Zealand,IMPORT,India,2022-09-01,Sep,2022,0.750,4.000


In [344]:
df_eu_nz[(df_eu_nz['Flow']=='IMPORT')]

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
68012,New Zealand,IMPORT,Australia,2010-01-01,Jan,2010,12.596,53.0
68013,New Zealand,IMPORT,Denmark,2010-01-01,Jan,2010,7.024,32.0
68014,New Zealand,IMPORT,Fiji,2010-01-01,Jan,2010,0.000,0.0
68015,New Zealand,IMPORT,France,2010-01-01,Jan,2010,0.046,0.0
68016,New Zealand,IMPORT,New Zealand,2010-01-01,Jan,2010,0.000,0.0
...,...,...,...,...,...,...,...,...
69078,New Zealand,IMPORT,Fiji,2022-09-01,Sep,2022,0.000,0.0
69079,New Zealand,IMPORT,France,2022-09-01,Sep,2022,0.000,0.0
69080,New Zealand,IMPORT,New Zealand,2022-09-01,Sep,2022,98.542,544.0
69081,New Zealand,IMPORT,India,2022-09-01,Sep,2022,0.750,4.0


In [345]:
df_nz[(df_nz['Flow']=='IMPORT')]

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
15912,New Zealand,IMPORT,Australia,2010-01-01,Jan,2010,12.596,53.0
15913,New Zealand,IMPORT,Denmark,2010-01-01,Jan,2010,7.024,32.0
15914,New Zealand,IMPORT,Fiji,2010-01-01,Jan,2010,0.000,0.0
15915,New Zealand,IMPORT,France,2010-01-01,Jan,2010,0.046,0.0
15916,New Zealand,IMPORT,New Zealand,2010-01-01,Jan,2010,0.000,0.0
...,...,...,...,...,...,...,...,...
16978,New Zealand,IMPORT,Fiji,2022-09-01,Sep,2022,0.000,0.0
16979,New Zealand,IMPORT,France,2022-09-01,Sep,2022,0.000,0.0
16980,New Zealand,IMPORT,New Zealand,2022-09-01,Sep,2022,98.542,544.0
16981,New Zealand,IMPORT,India,2022-09-01,Sep,2022,0.750,4.0


## 2.1 Plot

In [346]:
df_plot = df_eu_nz.copy() # Use deep copy to preserve the contents of df_eu 

In [347]:
df_plot = df_plot[(df_plot['Flow']=='EXPORT')]

In [348]:
df_plot.drop(
    ['Flow','Year','Month','Partner'],
    axis=1,
    inplace=True)


In [349]:
df_plot

,Country,Month Date,Quantity (tonnes),Value (k Euro)
0,Austria,2016-08-01,7.376,27.301
1,Austria,2011-01-01,0.043,0.100
2,Austria,2021-09-01,0.001,0.003
3,Austria,2020-11-01,0.000,0.017
4,Austria,2013-11-01,2.000,13.900
...,...,...,...,...
68007,New Zealand,2022-09-01,7.586,61.000
68008,New Zealand,2022-09-01,NaN,NaN
68009,New Zealand,2022-09-01,NaN,NaN
68010,New Zealand,2022-09-01,0.000,0.000


In [350]:
df_plot = df_plot.groupby(['Country','Month Date']).sum().reset_index()

In [351]:
df_plot

,Country,Month Date,Quantity (tonnes),Value (k Euro)
0,Austria,2010-01-01,7.529,33.448
1,Austria,2010-02-01,12.382,56.602
2,Austria,2010-03-01,7.520,34.685
3,Austria,2010-04-01,6.950,28.078
4,Austria,2010-05-01,0.698,3.211
...,...,...,...,...
3734,United Kingdom,2019-09-01,730.051,2947.043
3735,United Kingdom,2019-10-01,1212.479,4995.317
3736,United Kingdom,2019-11-01,1252.873,5139.480
3737,United Kingdom,2019-12-01,620.075,2575.875


In [352]:
# df_nz

In [353]:
# df_nz['Member State'] = 'New Zealand'
# df_nz['Value in thousand euro'] = 0

In [354]:
# df_nz

### Concatenate both sets together

In [355]:
# df_plot = pd.concat([df_plot,df_nz],ignore_index=True)

In [356]:
df_plot = df_plot.groupby([pd.Grouper(key='Month Date',freq='1Y'),'Country']).sum().reset_index()

In [357]:
df_plot

,Month Date,Country,Quantity (tonnes),Value (k Euro)
0,2010-12-31,Austria,78.552,356.651
1,2010-12-31,Belgium,9822.300,34837.812
2,2010-12-31,Bulgaria,80.475,219.644
3,2010-12-31,Czechia,869.249,3054.372
4,2010-12-31,Denmark,9798.743,44552.686
...,...,...,...,...
345,2022-12-31,Romania,176.248,1182.574
346,2022-12-31,Slovakia,53.646,393.132
347,2022-12-31,Slovenia,89.055,585.854
348,2022-12-31,Spain,1094.802,8036.446


In [358]:
df_plot.sort_values(by=['Month Date','Value (k Euro)'],ascending=False,inplace=True)

In [359]:
df_plot

,Month Date,Country,Quantity (tonnes),Value (k Euro)
342,2022-12-31,New Zealand,178756.266,1014677.000
335,2022-12-31,Ireland,68871.577,499476.726
331,2022-12-31,France,40360.689,298713.263
328,2022-12-31,Denmark,24279.438,154833.218
341,2022-12-31,Netherlands,17732.504,121849.368
...,...,...,...,...
2,2010-12-31,Bulgaria,80.475,219.644
10,2010-12-31,Hungary,21.858,93.619
9,2010-12-31,Greece,0.929,9.473
15,2010-12-31,Luxembourg,0.840,2.536


In [362]:
fig = px.line(df_plot, x="Month Date", y="Quantity (tonnes)", color='Country', title='Butter Exports')
fig.show()

In [361]:
fig = px.line(df_plot, x="Month Date", y="Value (k Euro)", color='Country', title='Butter Exports')
fig.show()